![title](../../images/datacraft-logo.png)

<h1><center>AI Act Day</center></h1>
<center><i>Datacraft workshop</i></center>


**Objective of the notebook**: understand how to tackle the fairness challenge in AI on real use case - HR recrutment

This exercise highly relies on the work conducted by the Dalex team (see [Dalex documentation](https://dalex.drwhy.ai/), [Fairness module in Dalex](https://dalex.drwhy.ai/python-dalex-fairness.html) or [Advanced tutorial on bias detection in Dalex](https://dalex.drwhy.ai/python-dalex-fairness2.html)). It is based on two [Stackoverflow survey](https://insights.stackoverflow.com/survey) (2021 & 2022) adapted to a recrutment issue.

Our humble ambition is to combine the power of different tools: Dalex as a base, complemented by other tools from the open source community, like [AIF360](https://aif360.mybluemix.net/).


<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Prerequesites" data-toc-modified-id="Prerequesites-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Prerequesites</a></span></li><li><span><a href="#Bias-a-priori" data-toc-modified-id="Bias-a-priori-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Bias a priori</a></span></li><li><span><a href="#Dataset" data-toc-modified-id="Dataset-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Dataset</a></span></li><li><span><a href="#EDA" data-toc-modified-id="EDA-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>EDA</a></span></li><li><span><a href="#ML-prerequisites" data-toc-modified-id="ML-prerequisites-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>ML prerequisites</a></span></li><li><span><a href="#Fairness-evaluation-principles" data-toc-modified-id="Fairness-evaluation-principles-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Fairness evaluation principles</a></span></li><li><span><a href="#Strategies" data-toc-modified-id="Strategies-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Strategies</a></span><ul class="toc-item"><li><span><a href="#Do-nothing" data-toc-modified-id="Do-nothing-7.1"><span class="toc-item-num">7.1&nbsp;&nbsp;</span>Do nothing</a></span><ul class="toc-item"><li><span><a href="#Training" data-toc-modified-id="Training-7.1.1"><span class="toc-item-num">7.1.1&nbsp;&nbsp;</span>Training</a></span></li><li><span><a href="#Algorithmic-performance" data-toc-modified-id="Algorithmic-performance-7.1.2"><span class="toc-item-num">7.1.2&nbsp;&nbsp;</span>Algorithmic performance</a></span></li><li><span><a href="#Fairness-performance" data-toc-modified-id="Fairness-performance-7.1.3"><span class="toc-item-num">7.1.3&nbsp;&nbsp;</span>Fairness performance</a></span></li></ul></li><li><span><a href="#Remove-sensitive-attribute" data-toc-modified-id="Remove-sensitive-attribute-7.2"><span class="toc-item-num">7.2&nbsp;&nbsp;</span>Remove sensitive attribute</a></span><ul class="toc-item"><li><span><a href="#Training" data-toc-modified-id="Training-7.2.1"><span class="toc-item-num">7.2.1&nbsp;&nbsp;</span>Training</a></span></li><li><span><a href="#Algorithmic-performance" data-toc-modified-id="Algorithmic-performance-7.2.2"><span class="toc-item-num">7.2.2&nbsp;&nbsp;</span>Algorithmic performance</a></span></li><li><span><a href="#Fairness-performance" data-toc-modified-id="Fairness-performance-7.2.3"><span class="toc-item-num">7.2.3&nbsp;&nbsp;</span>Fairness performance</a></span></li></ul></li><li><span><a href="#Adversarial-inprocessing" data-toc-modified-id="Adversarial-inprocessing-7.3"><span class="toc-item-num">7.3&nbsp;&nbsp;</span>Adversarial inprocessing</a></span></li><li><span><a href="#Calibrate-equalized-ODTS" data-toc-modified-id="Calibrate-equalized-ODTS-7.4"><span class="toc-item-num">7.4&nbsp;&nbsp;</span>Calibrate equalized ODTS</a></span></li><li><span><a href="#Comparison" data-toc-modified-id="Comparison-7.5"><span class="toc-item-num">7.5&nbsp;&nbsp;</span>Comparison</a></span></li></ul></li></ul></div>

## Prerequesites

In [ ]:
# We strongly recommend to install the necessary libraries from a dedicated virtual environment
# See Minicoda e.g.: https://docs.conda.io/projects/conda/en/latest/user-guide/tasks/manage-environments.html#creating-an-environment-with-commands

# ! pip install fairlearn
# ! pip install dalex -U
# ! pip install -U scikit-learn
# ! pip install -U pandas
# ! pip install - U aif360 
# ! pip install -U plotly

In [ ]:
from sklearn.compose import make_column_transformer, make_column_selector
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.tree import DecisionTreeClassifier

import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import sklearn
import dalex as dx

from copy import copy

{
    "numpy": np.__version__,
    "pandas": pd.__version__,
    "matplotlib": matplotlib.__version__,
    "seaborn": sns.__version__,
    "sklearn": sklearn.__version__,
    "dalex": dx.__version__,
}

In [ ]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:
sklearn.set_config(display="diagram")

## Bias a priori

*When implementing an AI system, fairness and biases must be an important component during conception, especially when dealing with sensitive information, and/or Personally Identifiable Information (PII), and/or Personal Health Information (PHI). Indeed, not only those information are bound to the law (GDPR in Europe e.g.), but they are also bound to a brand image challenge.*

Today's example aims at **assigning a risk with recruitment data**.

Before implementing any AI system to predict the likelihood of a candidate to be hired, **AI engineers AND business stakeholders** should:

- Sit and identify potential sources of biases
- Define one or several metrics that will quantify the bias of the AI system

![fairness_tree](../../images/fairness_tree.png)

## Dataset

StackOverflow's annual user-generated survey (over 70,000 responses from over 180 countries) of developers examines all aspects of the developer experience, from learning code to preferred technologies, version control and work experience. From the survey results, we have built a dataset with the following columns.

The columns of the dataset are :
- **Age**: age of the applicant, >35 years old or <35 years old *(categorical)*
- **EdLevel**: education level of the applicant (Undergraduate, Master, PhD...) *(categorical)*
- **Gender**: gender of the applicant, (Man, Woman, or NonBinary) *(categorical)*
- **MainBranch**: whether the applicant is a profesional developer *(categorical)*
- **YearsCode**: how long the applicant has been coding *(integer)*
- **YearsCodePro**: how long the applicant has been coding in a professional context, *(integer)*
- **PreviousSalary**: the applicant's previous job salary *(float)*
- **ComputerSkills**: number of computer skills known by the applicant *(integer)*
- **Employed**: target variable, whether the applicant has been hired *(categorical)*


In [ ]:
df = pd.read_csv('stackoverflow.csv', index_col=0)
target = "Employed"

In [ ]:
np.random.seed(2022)
df.sample(10).T

A priori, the sensitive variables from a recruitment perspective are Age and Gender. We first investigate this intuition with an exploratory data analysis.

## Exploratory Data Analysis

#### Visualization of data columns

In [ ]:
cmap = matplotlib.cm.Blues  # each subplot has its own blue color

fig, ax = plt.subplots(3,3, figsize = (12,8))

# Age
_ = df.Age.value_counts().plot.bar(subplots=True,
                                   ax=ax[0, 0],
                                   rot=1,
                                   color=cmap(0.2))
# Gender
_ = df.Gender.value_counts().plot.bar(subplots=True,
                                      ax=ax[0, 1],
                                      rot=1,
                                      color=cmap(0.3))
# MainBranch
_ = df.MainBranch.value_counts().plot.bar(subplots=True,
                                          ax=ax[0, 2],
                                          rot=1,
                                          color=cmap(0.4))
# EdLevel
_ = df.EdLevel.value_counts().plot.barh(subplots=True,
                                        ax=ax[1, 0],
                                        color=cmap(0.5))

# Employed
_ = df.Employed.value_counts().plot.bar(subplots=True,
                                        ax=ax[1, 1],
                                        rot=1,
                                        color=cmap(0.6))

# ComputerSkills
_ = df.ComputerSkills.clip(None, 30).plot.hist(subplots=True,
                                               ax= ax[1, 2],
                                               rot=1,
                                               color=cmap(0.7), 
                                               alpha=0.9, 
                                               edgecolor='w')

ax[1, 2].title.set_text('ComputerSkills')

# YearsCode
_ = df.YearsCode.plot.hist(subplots=True,
                                ax= ax[2, 0],
                                rot=1,
                                color=cmap(0.8),
                                alpha=0.9, 
                                edgecolor='w')

ax[2, 0].title.set_text('YearsCode')

# YearsCodePro
_ = df.YearsCodePro.plot.hist(subplots=True,
                              ax= ax[2, 1],
                              rot=1,
                              color=cmap(0.9),
                              alpha=0.9, 
                              edgecolor='w')

ax[2, 1].title.set_text('YearsCodePro')

# PreviousSalary
_ = df.PreviousSalary.plot.hist(subplots=True,
                                ax= ax[2, 2],
                                rot=1,
                                color=cmap(1.0),
                                alpha=0.9, 
                                edgecolor='w')

ax[2, 2].title.set_text('PreviousSalary')

fig.tight_layout()

#### Visualization of outliers

In [ ]:
sns.boxplot(df[["ComputerSkills", "YearsCode", "YearsCodePro"]], palette=[cmap(0.7), cmap(0.8), cmap(0.9)]);

In [ ]:
sns.boxplot(df["PreviousSalary"], palette=[cmap(1.0)]);

#### Visualization of gender bias on Employed

In [ ]:
var = (df
 .groupby("Gender")[target]
 .value_counts(normalize=True)
 .multiply(100)
 .round(1)
 .rename("%")
 .reset_index()
)

sns.barplot(data=var, ci=None, x="Gender", y="%", hue="Employed", width=0.5, palette=["orangered", "forestgreen"]);

#### Visualization of age bias on Employed

In [ ]:
var = (df
 .groupby("Age")[target]
 .value_counts(normalize=True)
 .multiply(100)
 .round(1)
 .rename("%")
 .reset_index()
)

sns.barplot(data=var, ci=None, x="Age", y="%", hue="Employed", width=0.5, palette=["orangered", "forestgreen"]);

#### Visualization of gender and age bias on Employed

In [ ]:
var = (df
 .groupby(["Gender", "Age"])[target]
 .value_counts(normalize=True)
 .multiply(100)
 .round(1)
 .rename("%")
 .reset_index()
)

sns.catplot(data=var,
            kind="bar",
            col="Age",
            row="Gender",
            x="Employed",
            y="%",
            palette=["orangered", "forestgreen"],
            height=2,
            aspect=1.5,
            margin_titles=True);

#### Analytics on biases for Gender and Age

In [ ]:
df["Gender"].value_counts()

In [ ]:
(df
 .groupby("Gender")[target]
 .value_counts(normalize=True)
 .multiply(100)
 .round(1)
 .to_frame()
)

In [ ]:
df["Age"].value_counts()

In [ ]:
(df
 .groupby("Age")[target]
 .value_counts(normalize=True)
 .multiply(100)
 .round(1)
 .to_frame()
)

In [ ]:
pd.crosstab(df["Gender"], df["Age"])

In [ ]:
(df
 .groupby(["Gender", "Age"])[target]
 .value_counts(normalize=True)
 .multiply(100)
 .round(1)
 .to_frame()
)

**Quick EDA**

If we bin by gender, we realize that women are:
- Under represented in volume: 3,518 vs. 68,573   
- Biased from an employment perspective: 44.9% vs. 54.1%

If we bin by age, we realize that people over 35 are:
- Under represented in volume: 25,643 vs. 47,819
- Biased from an employment perspective: 51.6% vs. 54.7%


If we combine the two, biases in input data are amplified:
- Under represented in volume: 938 vs. 44,343
- Biased from an employment perspective: 41.0% vs. 55.2%

**Conclusion: the apriori looks confirmed and will need to be carefully handled during modelling.**

----

**Notes**

1. This is a toy example where biases are "straightforward" and well identified as "recurrent" social biases. However, it might not always be as easy to detect them. Additional sources might come from data history, selection bias, data incompleteness, unexpected sources of bias (column/attribute), ...
2. In this example, we identified biases related to representation in volume. If we had not been exposed to such discrepencies, namely having a balanced dataset, could we have concluded that biases would have been limited while modelling? Not so sure, see [this article](https://arxiv.org/pdf/1811.08489.pdf).

## ML prerequisites

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    df.drop(columns=target),
    df[target],
    test_size=0.3,
    random_state=42
)

## Fairness evaluation principles

![dalex](../../images/dalex_pipeline.jpg)

The main object of dalex is the `Explainer` container which wraps a **dataset** (features and target) and a **trained model**. 

Once the data and the model have been wrapped, one needs to fix **protected and privileged attributes**.

**Important note**: beware these choices correspond to an a priori understanding of the problem and could miss hidden flaws of the model. An interesting line of work would consist in conducting a kind of grid-search exploration for potential biases.

In [ ]:
# Protected attribute is 0 if a man or non binary and 0 if a woman plus the age

protected = (pd.Series(np.where(X_test["Gender"] == "Woman", '1', '0'), index=X_test.index) 
             + '_' 
             + X_test.Age)
protected_train = (pd.Series(np.where(X_train["Gender"] == "Woman", '1', '0').astype(str), index=X_train.index) 
                   + '_' 
                   + X_train.Age)

# Privileged population is men under 35 years old
privileged = '0_<35'

## Strategies

Following section intends to implement different strategies to mitigate the bias:
- No strategy implemented
- Pre-processing strategy: edit the data priori to fitting a model
- In-processing: change the way a model is trained, changing the loss function e.g.
- Post-processing: edit the predictions once a model has been fitted

### Do nothing

#### Training

In [ ]:
preprocessor = make_column_transformer(
      ("passthrough", make_column_selector(dtype_include=np.number)),
      (OneHotEncoder(handle_unknown="ignore", sparse=False), make_column_selector(dtype_include=object))
)

clf_decisiontree = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', DecisionTreeClassifier(max_depth=10, random_state=123))
])

In [ ]:
# clf_decisiontree.fit(df.drop(columns=[target]), df[target])
clf_decisiontree.fit(X_train, y_train)

In [ ]:
# exp_decisiontree = dx.Explainer(clf_decisiontree, df.drop(columns=[target]), df[target], verbose=False)
exp_decisiontree = dx.Explainer(clf_decisiontree, X_test, y_test, verbose=True)

#### Algorithmic performance

In [ ]:
exp_decisiontree.model_performance().result

#### Fairness performance

Quoting Dalex' tutorial:


> The idea is that ratios between scores of privileged and unprivileged metrics should be close to 1. The closer, the fairer. To relax this criterion a little bit, it can be written more thoughtfully:

> $$ \forall i \in \{a, b, ..., z\}, \quad \epsilon < \frac{metric_i}{metric_{privileged}} < \frac{1}{\epsilon}.$$

> Where the epsilon is a value between 0 and 1, it should be a minimum acceptable value of the ratio. On default, it is 0.8, which adheres to four-fifths rule (80% rule) often looked at in hiring, for example.
"

#### Metrics used


- **Equal opportunity ratio** computed from True positive rate (recall)

> This number describes the proportions of correctly classified positive instances.

> $TPR = \frac{TP}{P}$

- **Predictive parity ratio** computed from Positive predicted value (precision)

> This number describes the ratio of samples which were correctly classified as positive from all the positive predictions.

> $PPV = \frac{TP}{TP + FP}$

- **Accuracy equality ratio** computed from Accuracy

> This number is the ratio of the correctly classified instances (positive and negative) of all decisions.

> $ACC = \frac{TP + TN}{TP + FP + TN + FN}$

- **Predictive equality ratio** computed from False positive rate

> This number describes the share the proportion of actual negatives which was falsely classified as positive.

> $FPR = \frac{FP}{TP + TN}$

- **Statistical parity ratio** computed from Positive rate

> This number is the overall rate of positively classified instances, including both correct and incorrect decisions.

> $PR = \frac{TP + FP}{TP + FP + TN + FN}$

The method `model_fairness` returns a fairness object from which fairness evaluations can be conducted. Notice that every metrics inherited from the confusion matrix are computed during the instantiation.

Two methods can then be performed:
- The `fairness_check` method, which returns a report on the fairness of the model. It requires an epsilon parameter that corresponds to the threshold ratio below which a given metric is considered to be unfair (default value is 0.8).
- The `plot` method, which allows to visualize the main fairness ratios between the protected subgroups and the privileged one.

In [ ]:
fairness_decisiontree = exp_decisiontree.model_fairness(protected=protected, privileged=privileged)

In [ ]:
fairness_decisiontree.fairness_check(epsilon = 0.8) # default epsilon

In [ ]:
fairness_decisiontree.plot(verbose=False)

**Notes**:
1. Fairness metrics work the exact same way as performance metrics do. If one was to fit a model on the entire dataset and foster overfitting (namely, skipping a `train_test_split` operation), she would end up with a non biased model.
2. A lots of metrics can be computed. It is important to define early in the conception which are the critical metrics to monitor

### Pre-processing: Remove sensitive attribute

The first thing that can come to mind is to remove sensitive variables. However, this option is considered as really naive since it can have no effect. 

**When may this method works ?**

➤ If the sensitive variable conveys the bias (mostly) on its own.

    ➤ This means, the sensitive variable is correlated with the target variable
    ➤ This also means, the sensitive variable is NOT correlated at all with other explanatory variables and any combination of other explanatory variables cannot be used as a proxi for the sensitive variable. 

Most of the time, the bias is shared accross explanatory variables. For example, a bias on gender may be present in many ones (salary, education, socio-professional category, etc.)

This transformation is not possible in the dalex module (since it not a recommended option to deal with bias). To implement it, a new model has to be trained and explained. This time the sensitive variable

#### Training

In [ ]:
# Retrain a model without sensitive variables "age" and "sex"
X_train_restricted = X_train.drop(['Gender', 'Age'], axis=1)

preprocessor_restr = make_column_transformer(
      ("passthrough", make_column_selector(dtype_include=np.number)),
      (OneHotEncoder(handle_unknown="ignore", sparse=False), make_column_selector(dtype_include=object))
)

clf_decisiontree_restr = Pipeline(steps=[
    ('preprocessor', preprocessor_restr),
    ('classifier', DecisionTreeClassifier(max_depth=7, random_state=123))
])

clf_decisiontree_restr.fit(X_train_restricted, y_train)

#### Algorithmic performance

In [ ]:
# Create a new dalex explainer for the model without sensitive variables
exp_decisiontree_restr = dx.Explainer(clf_decisiontree_restr, X_test, y_test, verbose=True)

exp_decisiontree_restr.model_performance().result

**Note:**
1. Here are the performance metrics for the new model. Results are quite similar to those of the model with all variables. However, considering we want to unbias results, it's quite difficult to use these metrics to compare models. Indeed, having a 100% precision on predicting a bias decision is far from our goal event if the model is perfect (at its predicting job).

#### Fairness performance
    

In [ ]:
# Let's see effect of removing sensitive variables on bias metrics
fairness_decisiontree_restr = exp_decisiontree_restr.model_fairness(protected=protected, privileged=privileged, 
                                                                    label='DecisionTreeClassifier_no_sensitive')

fairness_decisiontree_restr.fairness_check(epsilon = 0.8) # default epsilon

fairness_decisiontree_restr.plot()

Without a real surprise, the effect of removing sensitive variables did not unbias our results. In dalex it's also possible to compare models performances (according to bias metrics). Below, there is the comparison between the decision tree with and without sensitive variables :

In [ ]:
# Compare 2 (or more) fairness objects in dalex (add them as list in parameters). 
# It's also possible to choose the plot type ! 
fairness_decisiontree_restr.plot([fairness_decisiontree], type='radar')

### [Optional] Explaining the classifier's decisions

Before looking into bias mitigation techniques, we can try to interpret the classifier's predictions with explainability techniques. In our case, we have used a [decision tree](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html#sklearn.tree.DecisionTreeClassifier) and can therefore look at feature importances, or even plot the full tree.

In [ ]:
importances = clf_decisiontree['classifier'].feature_importances_

decisiontree_importances = pd.Series(importances, index = clf_decisiontree['preprocessor'].get_feature_names())

decisiontree_importances.plot.barh(title = 'Feature importances in Decisiontree',
                                   color = 'lightseagreen')

fig.tight_layout()

Despite the biases observed previously, the age and gender variables do not seem important for the decision tree's predictions. The most import feature is the number of skills of the applicant. One explanation for the biases may be that the number of skills is correlated to age or gender, which we investigate in the plots below.

In [ ]:
sns.catplot(data=df,
            kind="box",
            x="Age",
            y="ComputerSkills");

In [ ]:
sns.catplot(data=df,
            kind="box",
            x="Gender",
            y="ComputerSkills");

#### Exercise

Train a decision tree with a small depth, which allows toplot the entire tree and have an explainable model. Can you see some biases in the tree's decision path?

Now, let's check more appropriate ways to remove bias

### Pre-processing: Resampling

Dalex provide 2 types of resampling methods and 1 reweighting method. In this tutorial only the basic resampling is showed.

#### Training

In [ ]:
from dalex.fairness import resample
clf_resampled = copy(clf_decisiontree) # Create a copy to not alter the main object

# Resampling observations
indices_uniform = resample(protected_train, y_train, verbose = False)

# Re-fit model with resampled data
clf_resampled.fit(X_train.reset_index(drop=True).iloc[indices_uniform, :], y_train.reset_index(drop=True)[indices_uniform])


In [ ]:
exp_decisiontree_resampled = dx.Explainer(clf_resampled, X_test, y_test, verbose=True)

#### Algorithmic performance

In [ ]:
exp_decisiontree_resampled.model_performance().result

#### Fairness performance

In [ ]:
fairness_decisiontree_resampled = exp_decisiontree_resampled.model_fairness(
    protected, privileged, label='DecisionTreeClassifier_resampled')

fairness_decisiontree_resampled.fairness_check(epsilon = 0.8)


__Compare performance of the first model and the resampled one (visually)__

In [ ]:
fairness_decisiontree.plot([fairness_decisiontree_resampled])

The resampling method is partly random but it should increase the fairness of outputs at least on few fairness metrics.

In [ ]:
fairness_decisiontree.plot([fairness_decisiontree_resampled, fairness_decisiontree_restr], type='radar')

This resampling method seems to have succeeded at removing biases for our chosen threhsold.

### In-processing: Adversarial training

The adversarial inprocessing method consists in learning a target attribute $y$ (here the risk) while forgetting a fixed sensitive attribute $s$. This is done by learning a neural network and minimizing a loss of the form:
$$ \mathcal{L} = \mathcal{L}_{CE}(y,\hat{y}) - \lambda \mathcal{L}_{CE}(s,\hat{s}), $$
where $\lambda$ controls the fairness-accuracy tradeoff. 

This method is implemented in aif360 in the case of a binary sensitive attribute. In the following we incorporate it into the dalex pipeline.

For simplicity, we only investigate biases with respect to the variable Gender.

In [ ]:
from aif360.sklearn.inprocessing import AdversarialDebiasing
import tensorflow.compat.v1 as tf
sess = tf.Session()
tf.disable_eager_execution()
tf.random.set_random_seed(42)

In [ ]:
X_train["Gender"] = np.where(X_train["Gender"] == "Woman", 1, 0).astype(np.int64)
X_test["Gender"] = np.where(X_test["Gender"] == "Woman", 1, 0).astype(np.int64)

In [ ]:
preprocessor = make_column_transformer(
      ("passthrough", ['YearsCode', 'YearsCodePro', 'ComputerSkills', 'Gender']),
      (StandardScaler(), ['PreviousSalary']),
      (OneHotEncoder(handle_unknown="ignore"), make_column_selector(dtype_include=object)),
)

protected_adv = X_test["Gender"].astype(str)
privileged_adv = '0'

X_train_prep = preprocessor.fit_transform(X_train)
columns_names = preprocessor.get_feature_names_out(preprocessor.feature_names_in_)

class ToFrame():
    def __init__(self):
        pass
        #print('ok')
    
    def fit(self, arr, y=None):
        return self
    
    def transform(self, arr, y=None):
        df = pd.DataFrame(arr)
        df.columns = columns_names
        df.index = df['passthrough__Gender']
        return df

In [ ]:
adv_model = Pipeline(
    steps=[
        ('preprocessor', preprocessor),
        ('toframe', ToFrame()),
        ('adv', AdversarialDebiasing(prot_attr=['passthrough__Gender'], 
                                     debias=True,
                                     verbose=False,
                                     num_epochs=20,
                                     adversary_loss_weight=1e-2,
                                     random_state=42,
                                     classifier_num_hidden_units=512))
    ]
)        

#### Algorithmic performance

In [ ]:
# Warning: This cell may take some time to run!
adv_model.fit(X_train, y_train)

#### Algorithmic performance

In [ ]:
exp_adv_model = dx.Explainer(adv_model, X_test, y_test, verbose=True)
exp_adv_model.model_performance().result

#### Fairness performance

In [ ]:
fairness_adv_model = exp_adv_model.model_fairness(protected=protected_adv, privileged=privileged_adv, label="AdversarialTraining")
fairness_adv_model.fairness_check()

### Comparison to the decision tree


In [ ]:
fairness_decisiontree_gender = exp_decisiontree.model_fairness(
    protected=protected_adv, privileged=privileged_adv, label="DecisionTree")
fairness_decisiontree_gender.fairness_check()

In [ ]:
fairness_decisiontree_gender.plot([fairness_adv_model], type='radar')

### Post-processing: ROC-pivot

#### After-Training

For this method, there is no re-training to do since it's a post-processing method. The idea is to alter results in favor / defavor of some groups to increase the fairness metrics scores (privileged group VS others).

From a math point of view, 

Let, 
* `P` be the probability output of a model (higher probability means higher chances to get the favorable outcome, "1" in out case).
* `cutoff` be the value to assign values to 0 (below cutoff) or 1 (above cutoff)
* `𝜃` be the margin parameter to alter results (it is representing the notion of "close enough")
* `Priviledge` be the boolean value if the observation is part of the priviledge group

The roc pivot method will distinguish two cases : 

* The first one: if `|P - cutoff| < 𝜃 AND Priviledge AND P > cutoff` is `True` then the new probability became `P = cutoff - (P - cutoff)` which is now below the cutoff.

* The second case: if `|P - cutoff| < 𝜃 AND NOT(Priviledge) AND cutoff > P` is `True`, then the new probability became `P = cutoff + (cutoff - P)` which is above the cutoff value.


In [ ]:
from dalex.fairness import roc_pivot
exp_decisiontree_roc = copy(exp_decisiontree)

# Results modifications. Theta arbitrarily set at 0.1
exp_decisiontree_roc = roc_pivot(exp_decisiontree, protected, privileged, 
                                 theta = 0.1, verbose = False)

#### Algorithmic performance

In [ ]:
exp_decisiontree_roc.model_performance().result

#### Fairness performance

In [ ]:
fairness_decisiontree_roc = exp_decisiontree_roc.model_fairness(
    protected, 
    privileged, 
    label='DecisionTreeClassifier_roc')

fairness_decisiontree_roc.fairness_check(epsilon = 0.8)

Based on the fairness report, roc seems to have less effect on mitigating biais for this model.

__Compare performance of the first model and the resampled one (visually)__


In [ ]:
fairness_decisiontree.plot(
    [fairness_decisiontree_roc, 
     fairness_decisiontree_resampled, 
     fairness_decisiontree_restr])

In [ ]:
fairness_decisiontree.plot(
    [fairness_decisiontree_roc, 
     fairness_decisiontree_resampled, 
     fairness_decisiontree_restr], 
    type='radar')

## Exercices

Now that you have seen several bias mitigation techniques, it is now your turn to play with the data. We list below several tasks that you may tackle, either on the dataset used so far or on another more involved one.

### On this dataset

* Change hyperparameters of the decision tree (typically, its depth) to optimize the accuracy and investigate whether there is a tradeoff between accuracy and fairness. You can also try other classifiers such as random forests.

* Try other resampling or reweighting methods from Dalex to see if they perform better or worse for mitigating the biases. See the dalex [documentation](https://dalex.drwhy.ai/python/api/fairness/index.html) for help.


### On another more complete dataset

The dataframe `df_full` was also built from the Stackoverflow survey but contains more columns, specifically:
- **MentalHealth**: whether the applicant has mental health issues *(categorical)*
- **Accessibility**: whether the applicant has accessibility issues *(categorical)*
- **Country**: country of origin of the applicant *(categorical)*
- **HaveWorkedWith**: list of computer languages known by the applicant (note that the variable ComputerSkills is tthen he number of semicolon-separated skills) *(strings separated by semicolons)*

This is now your turn to fully investigate this dataset: idenfity potential sources of biases, do some exploratory data analysis, build a classifier to predict the target variable Employed, check its biases and mitigate them with your favorite tools.

In [ ]:
df_full = pd.read_csv('stackoverflow_full.csv', index_col=0)

In [ ]:
df_full.sample(10).T